# Setting

In [1]:
from utility import *

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
def RMSE(y, y_pred):
    return mean_squared_error(y, y_pred)**0.5


def train_model(train_data, target_data, model=LinearRegression()):  # baseline model : LInearRegression
    x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, random_state=0)

    model.fit(x_train, y_train)
    print("Model Training Complete!")

    pred_train, pred_test = model.predict(x_train), model.predict(x_test)
    
    plt.figure(figsize=(10, 8))
#     plt.scatter(pred_train, y_train, s=10)
    sns.regplot(pred_train, y_train, color='g')
    plt.xlabel("Predicted price")
    plt.ylabel("Actual price")
    plt.show()

    # cvs = cross_val_score(model, x_test, y_test, cv = 5)
    # print(">> cross_val_score mean =", cvs.mean())
    print(">> RMSE train =", RMSE(y_train, pred_train))
    print(">> RMSE validation =", RMSE(y_test, pred_test))
    print(">> MAE train =", mean_absolute_error(pred_train, y_train))
    print(">> MAE validation =", mean_absolute_error(pred_test, y_test))
    print("-------------------------------------------------")
    
    return model

    
def print_importance(model, df, added_columns):
    importance = model.coef_
    fs_data = []
    for i, x in enumerate(importance):
        fs_data.append([abs(x), df.columns[i]])
    fs_data.sort(key=lambda x: x[0], reverse=True)
   
    # 추가한 컬럼의 중요도
    for i in range(len(fs_data)):
        if fs_data[i][1] in added_columns:
            print(fs_data[i][1], ":", fs_data[i][0] ,">", i, "순위")
    print("-------------------------------------------------")
    print("총", len(fs_data) , "개")
    
    return fs_data

In [4]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from scipy.stats import randint

In [5]:
root = os.path.join(os.getcwd(), 'DATA')

# Import Data

In [6]:
df_train = pd.read_csv(os.path.join(root, 'preprocessed_train_notencoded.csv'))
df_weather_code = pd.read_csv(os.path.join(root, 'raw_weather_code.csv'), header=0, index_col=0)
weather_list = [pd.read_csv(os.path.join(root, 'raw_weather_20151228_20161227.csv'), encoding='cp949') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20161228_20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20171228_20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20181228_20191227.csv'), encoding='cp949') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20191228_20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20201228_20210818.csv'), encoding='euc-kr')]
df_exchange = pd.read_csv(os.path.join(root, 'preprocessed_exchange.csv'))
df_oil = pd.read_csv(os.path.join(root, 'preprocessed_oil.csv'))
df_weather_kr = pd.read_csv(os.path.join(root, 'preprocessed_weather_korea.csv'))
df_cpi = pd.read_csv(os.path.join(root, 'preprocessed_cpi.csv'))
df_weather_with_wf = pd.read_csv(os.path.join(root, 'df_weather_with_wf.csv')) # 3-2 전처리 가설 검증 df_weather_with_wf

In [7]:
final_squid = pd.read_csv(os.path.join(root, 'final_squid.csv'))
final_salmon = pd.read_csv(os.path.join(root, 'final_salmon.csv'))
final_whiteleg_shrimp = pd.read_csv(os.path.join(root, 'final_whiteleg_shrimp.csv'))

# Best Model Selection

In [8]:
def model_scaler(data, col, scaler = None):
  
  '''
  정규화 함수
  data : dataframe
  column : P_PRICE
  scaler : standard, robust, minmax, log

  '''
 
  features = data.drop(col, axis=1)
  target = data[col]

  if scaler == 'standard':
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    return features, target

  elif scaler == 'robust':
    scaler = RobustScaler()
    features = scaler.fit_transform(features)

    return features, target

  elif scaler == 'minmax':
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)

    return features, target

  elif scaler == 'log':
    features = np.log1p(features)

    return features, target

  elif scaler == 'None':

    return features, target


################################################################################################################################################

def model_train(data, col, scaler, model = None):

  '''
  
  data : dataframe
  column : P_PRICE
  scaler : standard, robust, minmax, log
  model_name : linear, ridge, lasso, elastic, decisiontree,
               randomforest, ada, gradient, xgb, lgbm

  '''

  features, target = model_scaler(data, col, scaler)
  x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)
  
  if model == 'linear': 
    
    model = LinearRegression()
    neg_mse_scores = cross_val_score(model, features, target, scoring = 'neg_mean_squared_error', cv = 10)
    rmse_scores = np.sqrt(-1 * neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)


    print('RMSE : {:.4f}'.format(avg_rmse))

  elif model == 'ridge':
    
    params = {
              'alpha': (0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1, 1, 10, 100, 200, 50, 30, 20, 29, 58),            
              'fit_intercept':(True, False),
              'normalize':(True, False),

              }

    ridge = Ridge(random_state=0)
    final = RandomizedSearchCV(ridge, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
    final.fit(x_train, y_train)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))
    
  elif model == 'lasso':

    params = {
              'alpha': (0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1, 1, 10, 100, 200, 50, 30, 20, 29, 58),            
              'fit_intercept':(True, False),
              'normalize':(True, False),

              }

    lasso = Lasso(random_state=0)
    final = RandomizedSearchCV(lasso, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
    final.fit(features, target)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))
  
  elif model == 'elastic':
    

    params = {
       'alpha': (0.1, 0.01, 0.5, 1, 3, 5, 10),
       'l1_ratio':(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.1, 0.1)
    }

    elastic = ElasticNet()
    final = RandomizedSearchCV(elastic, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
    final.fit(features, target)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))
  elif model == 'decisiontree':
    
    params = {
              'max_depth': [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],            
               #'min_child_samples': randint(5, 50),
              'min_samples_split':[1,3,5,7,10,15,20,25,30,45,50,60,70,80,90,100],
              'min_samples_leaf': [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              
    }

    dt = DecisionTreeRegressor(random_state=0)
    final = RandomizedSearchCV(dt, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
    final.fit(features, target)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))
  elif model == 'randomforest':
    
    params = {
              'max_depth': [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],           
              'n_estimators':[1,5,10,30,50,70,100,200,500,750,1000],
               #'min_child_samples': randint(5, 50),
              'min_samples_leaf':[1,3,5,7,10,20,30,50,70,100],
              'min_samples_split': [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              'max_leaf_nodes': [1,3,5,7,10,20,30,50,70,100,200,500,700,800,900,1000],

              }

    rf = RandomForestRegressor(random_state=0)
    final = RandomizedSearchCV(rf, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
    final.fit(features, target)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))


  elif model == 'gradinet':

    params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'subsample' : (0.01, 0.1, 0.5, 0.08, 0.35, 0.3, 0.001, 0.03, 0.006, 0.153, 0.193, 0.0012, 0.0083 ,1),
              'min_samples_split' : [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              'max_depth' : [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              }   

    grad = GradientBoostingRegressor()
    final = RandomizedSearchCV(grad, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
    final.fit(features, target)
    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

  elif model == 'xgb':
    
    params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' :  [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              'min_child_weight' :[0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              }   

    xgb = XGBRegressor()
    final = RandomizedSearchCV(xgb, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
    final.fit(features, target)

    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

  elif model == 'lgbm':
    params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' : [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              'min_child_weight' : [0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              'num_leaves': [1,3,5,7,9,15,25,50,100,200,300,500,700],
              'min_child_samples':[1,3,5,7,9,15,25,50,100,200,300,500,700],  
              }   

    lgbm = LGBMRegressor()
    final = RandomizedSearchCV(lgbm, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
    final.fit(features, target)

    pred = final.predict(x_test)

    print('Best Params:', final.best_params_)
    print('Best Score:', np.sqrt(-1 *final.best_score_))
    print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

In [9]:
lr = LinearRegression()

params = {
              'alpha': (0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1, 1, 10, 100, 200, 50, 30, 20, 29, 58),            
              'fit_intercept':(True, False),
              'normalize':(True, False),

              }

ridge = Ridge(random_state=0)
ridge = RandomizedSearchCV(ridge, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
   
    
params = {
              'alpha': (0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1, 1, 10, 100, 200, 50, 30, 20, 29, 58),            
              'fit_intercept':(True, False),
              'normalize':(True, False),

            }

lasso = Lasso(random_state=0)
lasso = RandomizedSearchCV(lasso, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
   
  
 
params = {
       'alpha': (0.1, 0.01, 0.5, 1, 3, 5, 10),
       'l1_ratio':(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.1, 0.1)
    }

elastic = ElasticNet()
elastic = RandomizedSearchCV(elastic, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)

    
params = {
              'max_depth': [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],            
               #'min_child_samples': randint(5, 50),
              'min_samples_split':[1,3,5,7,10,15,20,25,30,45,50,60,70,80,90,100],
              'min_samples_leaf': [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              
}

dt = DecisionTreeRegressor(random_state=0)
dt = RandomizedSearchCV(dt, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 1000, n_jobs = -1 ,random_state=0)
    

params = {
              'max_depth': [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],           
              'n_estimators':[1,5,10,30,50,70,100,200,500,750,1000],
               #'min_child_samples': randint(5, 50),
              'min_samples_leaf':[1,3,5,7,10,20,30,50,70,100],
              'min_samples_split': [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              'max_leaf_nodes': [1,3,5,7,10,20,30,50,70,100,200,500,700,800,900,1000],

              }

rf = RandomForestRegressor(random_state=0)
rf = RandomizedSearchCV(rf, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

   
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'subsample' : (0.01, 0.1, 0.5, 0.08, 0.35, 0.3, 0.001, 0.03, 0.006, 0.153, 0.193, 0.0012, 0.0083 ,1),
              'min_samples_split' : [1,3,5,7,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              'max_depth' : [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              }   

grad = GradientBoostingRegressor()
grad = RandomizedSearchCV(grad, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
 

    
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' :  [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              'min_child_weight' :[0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              }   

xgb = XGBRegressor()
xgb = RandomizedSearchCV(xgb, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)
   
    
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' : [1,3,5,7,9,15,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500,550,600,700,800,900,1000,1100,1200,1500],  
              'min_child_weight' : [0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              'num_leaves': [1,3,5,7,9,15,25,50,100,200,300,500,700],
              'min_child_samples':[1,3,5,7,9,15,25,50,100,200,300,500,700],  
              }   

lgbm = LGBMRegressor()
lgbm = RandomizedSearchCV(lgbm, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)


## Squid

In [10]:
model_train(final_squid,'P_PRICE','None','linear')

RMSE : 0.1504


In [11]:
model_train(final_squid,'P_PRICE','None','ridge')

Best Params: {'normalize': True, 'fit_intercept': True, 'alpha': 0.1}
Best Score: 0.12461289559596812
Predict RMSE: 0.16395828012218305


In [12]:
model_train(final_squid,'P_PRICE','None','lasso')

Best Params: {'normalize': True, 'fit_intercept': True, 'alpha': 0.0001}
Best Score: 0.1571647029703455
Predict RMSE: 0.16141134764055998


In [13]:
model_train(final_squid,'P_PRICE','None','elastic')

Best Params: {'l1_ratio': 0.0001, 'alpha': 0.01}
Best Score: 0.19448430765721741
Predict RMSE: 0.1802555221723289


In [14]:
model_train(final_squid,'P_PRICE','None','decisiontree')

Best Params: {'min_samples_split': 45, 'min_samples_leaf': 7, 'max_depth': 200}
Best Score: 0.18554653606317617
Predict RMSE: 0.14895550821111503


In [15]:
model_train(final_squid,'P_PRICE','None','randomforest')

Best Params: {'n_estimators': 70, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_leaf_nodes': 50, 'max_depth': 900}
Best Score: 0.1811504340147707
Predict RMSE: 0.0690766350174073


In [16]:
model_train(final_squid,'P_PRICE','None','gradinet')

Best Params: {'subsample': 1, 'n_estimators': 500, 'min_samples_split': 100, 'max_depth': 15, 'learning_rate': 0.1}
Best Score: 0.17620129798404802
Predict RMSE: 0.0015613023963505597


In [17]:
model_train(final_squid,'P_PRICE','None','xgb')

Best Params: {'n_estimators': 30, 'min_child_weight': 9, 'max_depth': 300, 'learning_rate': 0.351}
Best Score: 0.16511883322292426
Predict RMSE: 0.029415624763589603


In [18]:
model_train(final_squid,'P_PRICE','None','lgbm')

Best Params: {'num_leaves': 9, 'n_estimators': 500, 'min_child_weight': 1, 'min_child_samples': 7, 'max_depth': 9, 'learning_rate': 0.351}
Best Score: 0.1678278178093434
Predict RMSE: 1.605712197403056e-06


In [19]:
single_models = [ 
    ( 'linear_reg' , lr), 
    ( 'ridge' , ridge), 
    ( 'lasso' , lasso), 
    ( 'elasticnet' , elastic), 
    ( 'decisiontree' , dt), 
    ( 'randomforest' , rf),
    ( 'gradient' , grad),
    ( 'xgb' , xgb),
    ( 'lgbm' , lgbm),
]


squid_voting = VotingRegressor(single_models, n_jobs= -1)

features = final_squid.drop('P_PRICE', axis=1)
target = final_squid['P_PRICE']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

squid_voting.fit(x_train,y_train)

pred = squid_voting.predict(x_test) 

print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

Predict RMSE: 0.15644860944649747


Score와 RMSE가 모두 좋은 xgb 모델을 최종 모델로 선택


Voting 모델과 비교 예정

## Salmon

In [20]:
model_train(final_salmon,'P_PRICE','None','linear')

RMSE : 0.0996


In [21]:
model_train(final_salmon,'P_PRICE','None','ridge')

Best Params: {'normalize': True, 'fit_intercept': False, 'alpha': 0.003}
Best Score: 0.07729361267527128
Predict RMSE: 0.09405174930179241


In [22]:
model_train(final_salmon,'P_PRICE','None','lasso')

Best Params: {'normalize': True, 'fit_intercept': True, 'alpha': 0.01}
Best Score: 0.08829866482950782
Predict RMSE: 0.09756407193707708


In [23]:
model_train(final_salmon,'P_PRICE','None','elastic')

Best Params: {'l1_ratio': 0.5, 'alpha': 0.5}
Best Score: 0.08829866482950782
Predict RMSE: 0.09756407193707708


In [24]:
model_train(final_salmon,'P_PRICE','None','decisiontree')

Best Params: {'min_samples_split': 50, 'min_samples_leaf': 100, 'max_depth': 425}
Best Score: 0.0882986648295078
Predict RMSE: 0.09880300758006161


In [25]:
model_train(final_salmon,'P_PRICE','None','randomforest')

Best Params: {'n_estimators': 70, 'min_samples_split': 15, 'min_samples_leaf': 70, 'max_leaf_nodes': 200, 'max_depth': 50}
Best Score: 0.08829604112969916
Predict RMSE: 0.09741355196009162


In [26]:
model_train(final_salmon,'P_PRICE','None','gradinet')

Best Params: {'subsample': 0.0012, 'n_estimators': 500, 'min_samples_split': 70, 'max_depth': 25, 'learning_rate': 0.5}
Best Score: 0.0828484957685795
Predict RMSE: 0.10177177198824494


In [27]:
model_train(final_salmon,'P_PRICE','None','xgb')

Best Params: {'n_estimators': 70, 'min_child_weight': 5, 'max_depth': 700, 'learning_rate': 0.185}
Best Score: 0.08308510631019256
Predict RMSE: 0.006221793636250067


In [28]:
model_train(final_salmon,'P_PRICE','None','lgbm')

Best Params: {'num_leaves': 9, 'n_estimators': 500, 'min_child_weight': 1, 'min_child_samples': 7, 'max_depth': 9, 'learning_rate': 0.351}
Best Score: 0.08679137478138159
Predict RMSE: 9.383943228628539e-07


In [29]:
single_models = [ 
    ( 'linear_reg' , lr), 
    ( 'ridge' , ridge), 
    ( 'lasso' , lasso), 
    ( 'elasticnet' , elastic), 
    ( 'decisiontree' , dt), 
    ( 'randomforest' , rf),
    ( 'gradient' , grad),
    ( 'xgb' , xgb),
    ( 'lgbm' , lgbm),
]


salmon_voting = VotingRegressor(single_models, n_jobs= -1)

features = final_salmon.drop('P_PRICE', axis=1)
target = final_salmon['P_PRICE']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

salmon_voting.fit(x_train,y_train)

pred = salmon_voting.predict(x_test) 

print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

Predict RMSE: 0.07310787425498678


Score와 RMSE가 모두 좋은 xgb 모델을 최종 모델로 선택

Voting 모델과 비교 예정

## Whiteleg_shrimp

In [30]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','linear')

RMSE : 0.0686


In [31]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','ridge')

Best Params: {'normalize': True, 'fit_intercept': True, 'alpha': 0.096}
Best Score: 0.06430357908611088
Predict RMSE: 0.06155358109928035


In [32]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','lasso')

Best Params: {'normalize': True, 'fit_intercept': True, 'alpha': 0.0001}
Best Score: 0.06903210678435481
Predict RMSE: 0.05910512469864872


In [33]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','elastic')

Best Params: {'l1_ratio': 0.0001, 'alpha': 0.01}
Best Score: 0.07205399366192204
Predict RMSE: 0.05857675272056743


In [34]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','decisiontree')

Best Params: {'min_samples_split': 70, 'min_samples_leaf': 3, 'max_depth': 350}
Best Score: 0.06781885732759545
Predict RMSE: 0.04979353941227515


In [35]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','randomforest')

Best Params: {'n_estimators': 30, 'min_samples_split': 20, 'min_samples_leaf': 3, 'max_leaf_nodes': 1000, 'max_depth': 475}
Best Score: 0.06453594728752406
Predict RMSE: 0.03720533118564567


In [36]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','gradinet')

Best Params: {'subsample': 0.3, 'n_estimators': 200, 'min_samples_split': 35, 'max_depth': 1500, 'learning_rate': 0.008}
Best Score: 0.06782038468287582
Predict RMSE: 0.04740924026929726


In [37]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','xgb')

Best Params: {'n_estimators': 30, 'min_child_weight': 9, 'max_depth': 1200, 'learning_rate': 0.185}
Best Score: 0.07026569307523202
Predict RMSE: 0.024090612869304123


In [38]:
model_train(final_whiteleg_shrimp,'P_PRICE','None','lgbm')

Best Params: {'num_leaves': 3, 'n_estimators': 100, 'min_child_weight': 0.5, 'min_child_samples': 9, 'max_depth': 50, 'learning_rate': 0.01825}
Best Score: 0.06718620875185184
Predict RMSE: 0.04997575729838162


In [39]:
single_models = [ 
    ( 'linear_reg' , lr), 
    ( 'ridge' , ridge), 
    ( 'lasso' , lasso), 
    ( 'elasticnet' , elastic), 
    ( 'decisiontree' , dt), 
    ( 'randomforest' , rf),
    ( 'gradient' , grad),
    ( 'xgb' , xgb),
    ( 'lgbm' , lgbm),
]


whiteleg_shrimp_voting = VotingRegressor(single_models, n_jobs= -1)

features = final_whiteleg_shrimp.drop('P_PRICE', axis=1)
target = final_whiteleg_shrimp['P_PRICE']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

whiteleg_shrimp_voting.fit(x_train,y_train)

pred = whiteleg_shrimp_voting.predict(x_test) 

print('Predict RMSE:',(np.sqrt(mean_squared_error(y_test, pred))))

Predict RMSE: 0.05525640724766423


Score와 RMSE가 모두 좋은 xgb 모델을 최종 모델로 선택

Voting 모델과 비교 예정

## Best Model

### Squid

In [40]:
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' :  [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
              'min_child_weight' :[0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              }


xgb = XGBRegressor()
squid_final = RandomizedSearchCV(xgb, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

features = final_squid.drop('P_PRICE',axis=1)
target = final_squid['P_PRICE']

squid_final.fit(features, target)

RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100,...
                   param_distributions={'learning_rate': (0.01, 0.0001, 0.003,
                                                          0.5, 0.04, 0.008,
                                    

In [41]:
# params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
#               'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
#               'max_depth' : [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
#               'min_child_weight' : [0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'num_leaves': [1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'min_child_samples':[1,3,5,7,9,15,25,50,100,200,300,500,700],  
#               }   

# lgbm = LGBMRegressor()
# squid_final = RandomizedSearchCV(lgbm, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

# features = final_squid.drop('P_PRICE',axis=1)
# target = final_squid['P_PRICE']


# squid_final.fit(features, target)

### Salmon

In [42]:
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' :  [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
              'min_child_weight' :[0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              }   

xgb = XGBRegressor()
salmon_final = RandomizedSearchCV(xgb, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

features = final_salmon.drop('P_PRICE',axis=1)
target = final_salmon['P_PRICE']

salmon_final.fit(features, target)

RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100,...
                   param_distributions={'learning_rate': (0.01, 0.0001, 0.003,
                                                          0.5, 0.04, 0.008,
                                    

In [43]:
# params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
#               'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
#               'max_depth' : [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
#               'min_child_weight' : [0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'num_leaves': [1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'min_child_samples':[1,3,5,7,9,15,25,50,100,200,300,500,700],  
#               }   

# lgbm = LGBMRegressor()
# salmon_final = RandomizedSearchCV(lgbm, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

# features = final_salmon.drop('P_PRICE',axis=1)
# target = final_salmon['P_PRICE']


# salmon_final.fit(features, target)

### Whiteleg Shrimp

In [44]:
params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
              'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
              'max_depth' :  [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
              'min_child_weight' :[0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
              }   

xgb = XGBRegressor()
whiteleg_shrimp_final = RandomizedSearchCV(xgb, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

features = final_whiteleg_shrimp.drop('P_PRICE',axis=1)
target = final_whiteleg_shrimp['P_PRICE']

whiteleg_shrimp_final.fit(features, target)

RandomizedSearchCV(cv=10,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100,...
                   param_distributions={'learning_rate': (0.01, 0.0001, 0.003,
                                                          0.5, 0.04, 0.008,
                                    

In [45]:
# params = {'n_estimators' : [1,5,10,30,50,70,100,200,500,1000],
#               'learning_rate' :(0.01, 0.0001, 0.003, 0.5, 0.04, 0.008, 0.001, 0.351, 0.096, 0.853, 0.185, 0.01825, 0.012385, 0.1),
#               'max_depth' : [1,3,5,7,9,15,25,50,100,200,300,400,450,500,550,700,800,900,1000],  
#               'min_child_weight' : [0,0.05,0.5,1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'num_leaves': [1,3,5,7,9,15,25,50,100,200,300,500,700],
#               'min_child_samples':[1,3,5,7,9,15,25,50,100,200,300,500,700],  
#               }   

# lgbm = LGBMRegressor()
# whiteleg_shrimp_final = RandomizedSearchCV(lgbm, param_distributions = params, cv = 10, scoring = 'neg_mean_squared_error', n_iter = 50, n_jobs = -1 ,random_state=0)

# features = final_whiteleg_shrimp.drop('P_PRICE',axis=1)
# target = final_whiteleg_shrimp['P_PRICE']

# whiteleg_shrimp_final.fit(features, target)

# Test

## Test Data

### Import Data

In [127]:
df_test = pd.read_excel(os.path.join(root, 'test.xlsx'))

### Preprocessing

In [128]:
# test data용 
def check_week(df):
    """
    dataframe에 sdate 과 edate 사이에 모든 데이터가 있는지 확인하는 함수
    :param df: 검사하고자 하는 dataframe (set_week 형태)
    :return: 데이터가 전체 존재하는지 여부
    """
    cnt = 0
    sdate = date(2020, 1, 6)  # start date
    edate = date(2020, 12, 28)  # end date
    delta = edate - sdate  # as timedelta
    mem = set()

    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        year, week = day.isocalendar()[0], day.isocalendar()[1]
        if year * 100 + week in mem:
            continue
        mem.add(year * 100 + week)
        if df[(df['year'] == year) & (df['week'] == week)].empty:
            print((year, week), end="")
            cnt += 1
    if cnt > 0:
        print()
    print("missing", cnt, "values")

In [129]:
set_week(df_test, 'REG_DATE')

In [130]:
check_week(df_test)

missing 0 values


#### Squid

In [131]:
df_test_squid = df_test[(df_test['P_NAME']=='오징어') & ((df_test['CTRY_1']=='페루') | (df_test['CTRY_1']=='중국') | (df_test['CTRY_1']=='칠레')) & ((df_test['CTRY_2']=='페루') | (df_test['CTRY_2']=='중국') | (df_test['CTRY_2']=='칠레'))]

df_test_squid = df_test_squid[['CTRY_1','CTRY_2','P_PRICE','year','week']]

##### Merge Features

In [132]:
df_test_squid_add = df_test_squid.copy()

In [133]:
df_test_squid_add = pd.merge(df_test_squid_add, df_weather_with_wf, how='left', on=['year', 'week', 'CTRY_1'])

In [134]:
df_test_squid_add = pd.merge(df_test_squid_add, df_weather_kr, how='left', on=['year', 'week'])

In [135]:
df_test_squid_add = pd.merge(df_test_squid_add, df_oil, how='left', on=['year', 'week'])

In [136]:
df_test_squid_add = pd.merge(df_test_squid_add, df_cpi, how='left', on=['year', 'week'])

In [137]:
df_test_squid_add = pd.merge(df_test_squid_add, df_exchange, how='left', on=['year', 'week', 'CTRY_2'])

##### Filling Missing Values

In [138]:
df_test_squid_add.isna().sum() # 페루 날씨 데이터 결측값 존재

CTRY_1              0
CTRY_2              0
P_PRICE             0
year                0
week                0
rain              266
wind              266
temperature       266
wind_kr             0
temperature_kr      0
water_temp_kr       0
oil                 0
cpi_total           0
cpi_fish            0
exchange            0
dtype: int64

In [139]:
df_test_squid_add = df_test_squid_add.sort_values('CTRY_1')

df_test_squid_add.fillna(method='ffill',inplace=True)  # 가까운 나라인 칠레 데이터로 대체

In [140]:
df_test_squid_add.isna().sum() # 결측값 처리 완료

CTRY_1            0
CTRY_2            0
P_PRICE           0
year              0
week              0
rain              0
wind              0
temperature       0
wind_kr           0
temperature_kr    0
water_temp_kr     0
oil               0
cpi_total         0
cpi_fish          0
exchange          0
dtype: int64

##### Grouping

In [141]:
df_test_squid_add = df_test_squid_add.groupby(['year','week']).mean()

In [142]:
df_test_squid_add['temp_kr'] = (df_test_squid_add['temperature_kr'] + df_test_squid_add['water_temp_kr'] + df_test_squid_add['wind_kr']) / 3
df_test_squid_add['cpi'] = df_test_squid_add['cpi_fish'] / df_test_squid_add['cpi_total']

In [143]:
df_test_squid_add.drop(columns=['temperature_kr', 'cpi_fish', 'water_temp_kr','wind_kr' ,'cpi_total'], axis=1, inplace=True)

##### Final Test DF

In [144]:
final_test_squid = df_test_squid_add

In [145]:
final_test_squid

P_PRICE      rain      wind  temperature        oil  exchange  \
year week                                                                   
2020 2     2.675438  1.854202  6.489202     9.172139  60.836000  0.137525   
     3     2.799003  1.193739  7.306846    10.025407  58.408333  0.173614   
     4     2.820781  2.338254  6.148527    11.269140  56.718000  0.149049   
     5     2.573288  1.365970  7.628200    12.503676  52.730000  0.179170   
     6     2.749292  1.538731  7.373124    12.883320  50.348000  0.257825   
     7     3.181970  1.213793  9.034483    15.417241  51.085000  0.294600   
     8     2.915586  1.131917  7.726606    12.581538  52.940000  0.265280   
     9     2.421211  1.289282  7.227372    13.671240  48.382000  0.162456   
     10    3.057574  1.629238  6.822567    12.562841  45.578000  0.209559   
     11    2.733139  1.907704  6.865263    14.030797  32.340000  0.181779   
     12    2.741034  1.485602  6.749179    13.832490  24.734000  0.141757   
     13    2.356233  3.277679  6.359435    13.316548  23.194000  0.121584   
     14    1.916776  1.674573  7.829853    12.860519  22.908000  0.108282   
     15    2.574854  1.822569  5.118715    12.456691  24.390000  0.096637   
     16    2.400193  3.023931  6.614963    13.460701  20.106000  0.125275   
     17    2.360139  2.759457  5.702737    12.604273   3.920000  0.144980   
     18    2.324205  2.873517  4.751807    13.102257  15.760000  0.102957   
     19    2.741465  4.143514  4.997552    13.640896  23.446000  0.131252   
     20    2.769251  2.647204  5.602269    14.587698  26.440000  0.157785   
     21    2.554303  3.944981  5.345824    14.104335  32.980000  0.143742   
     22    2.618530  5.068064  3.203662    16.851717  34.056000  0.161729   
     23    2.349213  4.092577  5.342365    14.466859  37.300000  0.127664   
     24    2.563945  4.148873  5.931791    15.741780  37.866000  0.156175   
     25    2.253062  4.513959  5.347447    15.746461  38.410000  0.128363   
     26    2.575724  5.701161  3.604829    16.077802  39.210000  0.094740   
     27    2.393032  4.885351  5.119784    15.625452  39.944000  0.126094   
     28    2.660586  5.094077  5.484832    15.825131  40.464000  0.152781   
     29    2.843682  3.261876  5.836132    14.537032  40.586000  0.143322   
     30    2.492329  3.840675  5.116900    14.982333  41.406000  0.127446   
     31    2.644042  3.135792  5.868393    15.350687  40.820000  0.166417   
     32    2.724842  3.032574  6.145607    15.089319  41.614000  0.167732   
     33    2.678793  2.872190  6.860268    14.337993  42.094000  0.184571   
     34    2.635599  2.678817  6.280159    14.464303  42.726000  0.166619   
     35    2.327621  3.547365  6.255224    14.337645  43.074000  0.155252   
     36    2.408983  2.577445  5.759892    14.644103  41.128333  0.175420   
     37    2.575534  2.415204  6.893904    13.840812  37.728000  0.170967   
     38    2.559725  2.393986  7.418194    14.372485  39.556000  0.167938   
     39    2.422035  1.761276  7.545050    14.436652  39.880000  0.192612   
     40    2.177672  2.414717  6.449929    13.035608  39.176000  0.155822   
     41    2.556555  1.757687  7.139676    13.730346  40.326000  0.187734   
     42    2.240754  2.404397  7.925579    14.182629  40.502000  0.191283   
     43    4.139414  2.008950  6.705283    14.324230  40.562000  0.204674   
     44    2.780823  1.742435  7.571535    15.508261  37.496000  0.248789   
     45    2.626703  1.323387  6.276375    13.625489  37.910000  0.181215   
     46    2.624950  4.242427  7.446761    15.135449  40.870000  0.207084   
     47    2.381523  1.963343  6.852907    12.250853  41.696000  0.193693   
     48    2.334349  1.368119  7.396289    13.088846  44.838000  0.245850   
     49    2.382441  1.128388  6.514284    10.984904  45.414000  0.230150   
     50    2.823628  1.493529  6.661796    11.425715  46.046000  0.230787   
     51    2.047650  1.382351  7.247595    12.234168  47.978000  0.218381   
     52 

#### Salmon

In [146]:
df_test_salmon = df_test[(df_test['P_NAME']=='연어') & (df_test['CTRY_1']=='노르웨이') & (df_test['CTRY_2']=='노르웨이')]

df_test_salmon = df_test_salmon[['CTRY_1','CTRY_2','P_PRICE','year','week']]

,CTRY_1,CTRY_2,P_PRICE,year,week
37,노르웨이,노르웨이,28.500000,2020,2
94,노르웨이,노르웨이,12.199457,2020,2
195,노르웨이,노르웨이,16.335035,2020,2
196,노르웨이,노르웨이,14.756580,2020,2
215,노르웨이,노르웨이,13.170679,2020,2
...,...,...,...,...,...
9435,노르웨이,노르웨이,8.750000,2020,53
9448,노르웨이,노르웨이,29.740000,2020,53
9459,노르웨이,노르웨이,15.290000,2020,53
9471,노르웨이,노르웨이,14.260000,2020,53


##### Merge Features

In [147]:
df_test_salmon_add = df_test_salmon.copy()

In [148]:
df_test_salmon_add = pd.merge(df_test_salmon_add, df_weather_with_wf, how='left', on=['year', 'week', 'CTRY_1'])

In [149]:
df_test_salmon_add = pd.merge(df_test_salmon_add, df_weather_kr, how='left', on=['year', 'week'])

In [150]:
df_test_salmon_add = pd.merge(df_test_salmon_add, df_oil, how='left', on=['year', 'week'])

In [151]:
df_test_salmon_add = pd.merge(df_test_salmon_add, df_cpi, how='left', on=['year', 'week'])

In [152]:
df_test_salmon_add = pd.merge(df_test_salmon_add, df_exchange, how='left', on=['year', 'week','CTRY_2'])

##### Filling Missing Values

In [153]:
df_test_salmon_add.isna().sum() # 날씨 데이터 결측값 존재

CTRY_1              0
CTRY_2              0
P_PRICE             0
year                0
week                0
rain              232
wind              232
temperature       232
wind_kr             0
temperature_kr      0
water_temp_kr       0
oil                 0
cpi_total           0
cpi_fish            0
exchange            0
dtype: int64

In [154]:
# 평균 값으로 대체

df_test_salmon_add['rain'].fillna(np.mean(df_test_salmon_add['rain']),inplace=True)
df_test_salmon_add['wind'].fillna(np.mean(df_test_salmon_add['wind']),inplace=True)
df_test_salmon_add['temperature'].fillna(np.mean(df_test_salmon_add['temperature']),inplace=True)

In [155]:
df_test_salmon_add.isna().sum() # 결측값 처리 완료

CTRY_1            0
CTRY_2            0
P_PRICE           0
year              0
week              0
rain              0
wind              0
temperature       0
wind_kr           0
temperature_kr    0
water_temp_kr     0
oil               0
cpi_total         0
cpi_fish          0
exchange          0
dtype: int64

##### Grouping

In [156]:
df_test_salmon_add = df_test_salmon_add.groupby(['year','week']).mean()

In [157]:
df_test_salmon_add['temp_kr'] = (df_test_salmon_add['temperature_kr'] + df_test_salmon_add['water_temp_kr'] + df_test_salmon_add['wind_kr']) / 3
df_test_salmon_add['cpi'] = df_test_salmon_add['cpi_fish'] / df_test_salmon_add['cpi_total'] 

In [158]:
df_test_salmon_add.drop(columns=['temperature_kr', 'cpi_fish', 'water_temp_kr','wind_kr', 'cpi_total'], axis=1, inplace=True)

##### Final Test DF

In [159]:
final_test_salmon = df_test_salmon_add

In [160]:
final_test_salmon

P_PRICE      rain      wind  temperature        oil  exchange  \
year week                                                                    
2020 2     16.992350  2.692562  7.074380     1.905372  60.836000    0.1137   
     3     17.340328  2.125000  6.288793    -0.349569  58.408333    0.1124   
     4     15.587532  2.182463  7.123134    -0.490299  56.718000    0.1120   
     5     16.892669  1.572358  5.772358    -1.876829  52.730000    0.1094   
     6     15.975454  2.724380  6.028926    -2.043802  50.348000    0.1077   
     7     15.170032  1.948305  6.830508    -0.350638  51.085000    0.1076   
     8     15.080865  1.690226  7.349624    -0.184962  52.940000    0.1080   
     9     15.723319  1.756667  5.105556    -4.196667  48.382000    0.1068   
     10    15.735015  2.389055  5.467662    -1.223383  45.578000    0.1076   
     11    15.187748  1.470787  6.400749    -2.719476  32.340000    0.1044   
     12    15.017789  1.778125  6.473958    -2.197396  24.734000    0.0973   
     13    14.675168  2.409589  7.123288    -0.349315  23.194000    0.0862   
     14    14.396876  1.864981  6.634241    -1.487160  22.908000    0.0948   
     15    15.218819  1.700424  6.622881     0.936017  24.390000    0.0958   
     16    12.040515  1.695798  6.378151     2.167647  20.106000    0.0966   
     17    14.279194  1.958000  5.700000     2.591333   3.920000    0.0955   
     18    13.881676  1.271515  3.890909     2.509091  15.760000    0.0951   
     19    13.452220  1.504891  5.744565     2.166848  23.446000    0.0967   
     20    13.567672  1.602500  6.862500     2.125625  26.440000    0.0968   
     21    14.117930  1.902564  5.196581     5.842735  32.980000    0.0997   
     22    14.430410  0.806838  4.623932     6.235897  34.056000    0.0994   
     23    15.218256  0.876190  4.142857     6.966667  37.300000    0.1039   
     24    15.704652  1.778858  5.921943     0.399247  37.866000    0.1080   
     25    15.732229  1.778858  5.921943     0.399247  38.410000    0.1048   
     26    14.820120  1.778858  5.921943     0.399247  39.210000    0.1041   
     27    14.637310  1.778858  5.921943     0.399247  39.944000    0.1033   
     28    14.746793  1.778858  5.921943     0.399247  40.464000    0.1064   
     29    14.494195  1.778858  5.921943     0.399247  40.586000    0.1059   
     30    14.202351  1.778858  5.921943     0.399247  41.406000    0.1084   
     31    15.263169  1.778858  5.921943     0.399247  40.820000    0.1099   
     32    14.266385  1.778858  5.921943     0.399247  41.614000    0.1095   
     33    14.027953  1.778858  5.921943     0.399247  42.094000    0.1106   
     34    14.434727  1.778858  5.921943     0.399247  42.726000    0.1129   
     35    14.297317  1.778858  5.921943     0.399247  43.074000    0.1111   
     36    14.017716  1.778858  5.921943     0.399247  41.128333    0.1146   
     37    14.295397  1.778858  5.921943     0.399247  37.728000    0.1119   
     38    14.276205  1.778858  5.921943     0.399247  39.556000    0.1107   
     39    14.575858  1.778858  5.921943     0.399247  39.880000    0.1077   
     40    15.085690  1.778858  5.921943     0.399247  39.176000    0.1054   
     41    14.927119  1.778858  5.921943     0.399247  40.326000    0.1085   
     42    14.598102  1.778858  5.921943     0.399247  40.502000    0.1094   
     43    14.698816  1.778858  5.921943     0.399247  40.562000    0.1072   
     44    19.295902  1.778858  5.921943     0.399247  37.496000    0.1082   
     45    14.480638  1.778858  5.921943     0.399247  37.910000    0.1046   
     46    14.481199  1.778858  5.921943     0.399247  40.870000    0.1105   
     47    14.269818  1.778858  5.921943     0.399247  41.696000    0.1104   
     48    14.583272  1.778858  5.921943     0.399247  44.838000    0.1106   
     49    14.545394  1.778858  5.921943     0.399247  45.414000    0.1124   
     50    14.501844  1.778858  5.921943     0.399247  46.046000    0.1143   
     51    14.571845  1.778858  5.9

#### Whiteleg Shrimp

In [161]:
df_test_whiteleg_shrimp = df_test[(df_test['P_NAME']=='흰다리새우') & ((df_test['CTRY_1']=='베트남') | (df_test['CTRY_1']=='태국')) & ((df_test['CTRY_2']=='베트남') | (df_test['CTRY_2']=='태국'))]

df_test_whiteleg_shrimp = df_test_whiteleg_shrimp[['CTRY_1','CTRY_2','P_PRICE','year','week']]

##### Merge Features

In [162]:
df_test_whiteleg_shrimp_add = df_test_whiteleg_shrimp.copy()

In [163]:
df_test_whiteleg_shrimp_add = pd.merge(df_test_whiteleg_shrimp_add, df_weather_with_wf, how='left', on=['year', 'week', 'CTRY_1'])

In [164]:
df_test_whiteleg_shrimp_add = pd.merge(df_test_whiteleg_shrimp_add, df_weather_kr, how='left', on=['year', 'week'])

In [165]:
df_test_whiteleg_shrimp_add = pd.merge(df_test_whiteleg_shrimp_add, df_oil, how='left', on=['year', 'week'])

In [166]:
df_test_whiteleg_shrimp_add = pd.merge(df_test_whiteleg_shrimp_add, df_cpi, how='left', on=['year', 'week'])

In [167]:
df_test_whiteleg_shrimp_add = pd.merge(df_test_whiteleg_shrimp_add, df_exchange, how='left', on=['year', 'week','CTRY_2'])

##### Filling Missing Values

In [168]:
df_test_whiteleg_shrimp_add.isna().sum()

CTRY_1            0
CTRY_2            0
P_PRICE           0
year              0
week              0
rain              0
wind              0
temperature       0
wind_kr           0
temperature_kr    0
water_temp_kr     0
oil               0
cpi_total         0
cpi_fish          0
exchange          0
dtype: int64

##### Grouping

In [169]:
df_test_whiteleg_shrimp_add = df_test_whiteleg_shrimp_add.groupby(['year','week']).mean()

In [170]:
df_test_whiteleg_shrimp_add['temp_kr'] = (df_test_whiteleg_shrimp_add['temperature_kr'] + df_test_whiteleg_shrimp_add['water_temp_kr'] + df_test_whiteleg_shrimp_add['wind_kr']) / 3
df_test_whiteleg_shrimp_add['cpi'] = df_test_whiteleg_shrimp_add['cpi_fish'] / df_test_whiteleg_shrimp_add['cpi_total'] 

In [171]:
df_test_whiteleg_shrimp_add.drop(columns=['temperature_kr', 'cpi_fish', 'water_temp_kr','wind_kr', 'cpi_total'], axis=1, inplace=True)

##### Final Test DF

In [172]:
final_test_whiteleg_shrimp = df_test_whiteleg_shrimp_add

In [173]:
final_test_whiteleg_shrimp

P_PRICE       rain      wind  temperature        oil  exchange  \
year week                                                                     
2020 2      9.902466   4.828017  1.581897    24.319655  60.836000  0.012439   
     3      9.595764   2.096739  1.454710    20.284783  58.408333  0.008307   
     4      9.313120   4.520435  1.649733    19.595531  56.718000  0.004737   
     5     10.731888   5.597820  1.524128    21.577209  52.730000  0.012214   
     6     11.815052   4.016703  2.059341    20.565128  50.348000  0.010762   
     7     11.837182   2.388179  2.716063    24.203563  51.085000  0.015971   
     8      9.060699   5.653000  2.816667    23.468333  52.940000  0.006454   
     9     10.900109   5.718000  2.146667    25.717000  48.382000  0.015822   
     10    11.752713  11.414582  1.835375    21.813991  45.578000  0.005319   
     11    13.278888   6.423730  2.350794    24.409365  32.340000  0.010629   
     12    12.167508   3.572596  1.478365    22.772837  24.734000  0.007807   
     13    12.074429   7.234326  2.202194    24.860188  23.194000  0.015271   
     14    11.699113   5.291375  1.576312    23.977782  22.908000  0.015321   
     15    12.336940   6.571178  2.154275    25.066388  24.390000  0.013096   
     16    12.885259   9.213038  1.860708    25.829463  20.106000  0.013139   
     17    10.531373   8.086518  2.172862    25.511672   3.920000  0.015421   
     18    12.741518   4.168062  1.823114    27.331878  15.760000  0.013224   
     19    10.975996   7.861725  2.185882    28.910431  23.446000  0.015471   
     20    12.433387  11.408472  1.896429    27.839187  26.440000  0.013310   
     21    12.992849   7.954939  2.020408    28.530367  32.980000  0.013396   
     22    12.863254  10.671229  1.757455    28.167494  34.056000  0.012546   
     23    12.298451   9.677091  1.639640    27.962613  37.300000  0.015821   
     24    11.462716  10.005641  2.125040    27.726149  37.866000  0.010629   
     25    11.543317  11.554198  1.827204    27.577127  38.410000  0.014335   
     26     9.772050   9.652292  1.463366    27.660929  39.210000  0.009259   
     27    11.568231   8.453973  2.078175    27.353845  39.944000  0.018533   
     28    13.310962   7.074311  1.620621    27.773540  40.464000  0.012866   
     29    11.317738   4.962020  2.104250    28.331896  40.586000  0.009117   
     30    10.571139   9.358282  1.767656    27.949645  41.406000  0.011802   
     31    11.521077  14.500183  2.723045    27.862996  40.820000  0.010629   
     32    10.593069  10.958704  1.901788    27.107347  41.614000  0.013739   
     33    11.227565  13.649860  1.512005    26.568587  42.094000  0.013782   
     34    10.196777   6.296310  1.441667    26.640000  42.726000  0.013782   
     35    11.652322   7.820372  1.430609    28.160963  43.074000  0.009088   
     36    11.038631  13.130540  1.639205    27.382623  41.128333  0.016122   
     37    11.637289  13.141535  1.176229    26.698575  37.728000  0.021281   
     38    10.616361  19.284710  2.408357    26.431814  39.556000  0.014202   
     39    12.426220  13.139945  1.387963    26.288724  39.880000  0.013696   
     40    10.879439  14.786775  1.394973    26.598999  39.176000  0.015822   
     41    11.382807  11.145656  1.963546    26.075885  40.326000  0.015972   
     42    11.968719  19.619419  2.146495    25.039011  40.502000  0.013782   
     43    10.888223   8.699306  1.370370    24.320602  40.562000  0.016022   
     44    11.467702  16.533004  1.991453    24.920085  37.496000  0.014246   
     45    12.732091   6.718846  2.256593    26.117857  37.910000  0.013782   
     46     9.438308  12.507702  2.499911    24.158474  40.870000  0.013106   
     47    11.865793  11.691111  1.766667    25.946667  41.696000  0.016572   
     48    12.399499  11.482479  2.058761    25.551709  44.838000  0.016472   
     49    10.708834  18.282692  3.076923    23.184615  45.414000  0.008282   
     50    10.943944  11.610982  0.951786    25.649196  46.0460

## evaluation

### Squid

In [90]:
squid_test_features = final_test_squid.drop('P_PRICE',axis=1)
squid_test_target = final_test_squid['P_PRICE']

pred = squid_final.predict(squid_test_features)

print('Best Params:', squid_final.best_params_)
print('Best Score:', np.sqrt(-1 *squid_final.best_score_))
print('Predict RMSE:',(np.sqrt(mean_squared_error(squid_test_target, np.exp(pred)-1))))

Best Params: {'n_estimators': 50, 'min_child_weight': 7, 'max_depth': 450, 'learning_rate': 0.1}
Best Score: 0.16562297026620973
Predict RMSE: 0.3362952952405674


In [91]:
pred = squid_voting.predict(squid_test_features)

print('Predict RMSE:',(np.sqrt(mean_squared_error(squid_test_target, np.exp(pred)-1))))

Predict RMSE: 47392.14468397102


RMSE이 더 작은 기존의 best 모델 선택

### Salmon

In [92]:
salmon_test_features = final_test_salmon.drop('P_PRICE',axis=1)
salmon_test_target = final_test_salmon['P_PRICE']

pred = salmon_final.predict(salmon_test_features)

print('Best Params:', salmon_final.best_params_)
print('Best Score:', np.sqrt(-1 *salmon_final.best_score_))
print('Predict RMSE:',(np.sqrt(mean_squared_error(salmon_test_target, np.exp(pred)-1))))

Best Params: {'n_estimators': 70, 'min_child_weight': 0.05, 'max_depth': 9, 'learning_rate': 0.185}
Best Score: 0.08598306647204754
Predict RMSE: 1.1852836932011916


In [93]:
pred = salmon_voting.predict(salmon_test_features)

print('Predict RMSE:',(np.sqrt(mean_squared_error(salmon_test_target, np.exp(pred)-1))))

Predict RMSE: 6.54080678536826


RMSE이 더 작은 기존의 best 모델 선택

### Whiteleg Shrimp

In [94]:
whiteleg_shrimp_test_features = final_test_whiteleg_shrimp.drop('P_PRICE',axis=1)
whiteleg_shrimp_test_target = final_test_whiteleg_shrimp['P_PRICE']

pred = whiteleg_shrimp_final.predict(whiteleg_shrimp_test_features)

print('Best Params:', whiteleg_shrimp_final.best_params_)
print('Best Score:', np.sqrt(-1 *whiteleg_shrimp_final.best_score_))
print('Predict RMSE:',(np.sqrt(mean_squared_error(whiteleg_shrimp_test_target,np.exp(pred)-1))))

Best Params: {'n_estimators': 30, 'min_child_weight': 0, 'max_depth': 1, 'learning_rate': 0.185}
Best Score: 0.06744433538097151
Predict RMSE: 0.9746104086328865


In [95]:
pred = whiteleg_shrimp_voting.predict(whiteleg_shrimp_test_features)

print('Predict RMSE:',(np.sqrt(mean_squared_error(whiteleg_shrimp_test_target, np.exp(pred)-1))))

Predict RMSE: 130.04227867528763


RMSE이 더 작은 기존의 best 모델 선택

# Result

In [96]:
# test data에 맞게 year,week filtering

oil_predict = df_oil[(df_oil['year']==2021) & (df_oil['week']<=26)].groupby(['year','week']).mean()
weather_kr_predict = df_weather_kr[(df_weather_kr['year']==2021) & (df_weather_kr['week']<=26)].groupby(['year','week']).mean()
cpi_predict = df_cpi[(df_cpi['year']==2021) & (df_cpi['week']<=26)].groupby(['year','week']).mean()

In [97]:
# 파생 변수 추가 및 기존 변수들 제거

weather_kr_predict['temp_kr'] = (weather_kr_predict['temperature_kr'] + weather_kr_predict['water_temp_kr'] + weather_kr_predict['wind_kr']) / 3

weather_kr_predict.drop(columns=['temperature_kr', 'wind_kr','water_temp_kr'], axis=1, inplace=True)

cpi_predict['cpi'] = cpi_predict['cpi_fish'] / cpi_predict['cpi_total'] 

cpi_predict.drop(columns=['cpi_fish', 'cpi_total'], axis=1, inplace=True)

## Merge Features

In [98]:
df_predict = oil_predict.copy()

df_predict = pd.merge(df_predict,weather_kr_predict,how='left',on=['year','week'])

In [99]:
df_predict = pd.merge(df_predict,cpi_predict,how='left',on=['year','week'])

### Squid

오징어 모델용 test features들을 생성. -> 최종 결과값 도출

#### squid test features

In [100]:
# 오징어 주요 제조국 및 수출국 filtering

squid_exchange_predict = df_exchange[(df_exchange['CTRY_2'] == '페루') | (df_exchange['CTRY_2'] == '중국') | (df_exchange['CTRY_2'] == '칠레')]
squid_weather_predict = df_weather_with_wf[(df_weather_with_wf['CTRY_1'] == '페루') | (df_weather_with_wf['CTRY_1'] == '중국') | (df_weather_with_wf['CTRY_1'] == '칠레')]

In [101]:
# test data에 맞게 year,week filtering

squid_exchange_predict = squid_exchange_predict[(squid_exchange_predict['year']==2021) & (squid_exchange_predict['week']<=26)]
squid_weather_predict = squid_weather_predict[(squid_weather_predict['year']==2021) & (squid_weather_predict['week']<=26)]

In [102]:
# grouping

squid_exchange_predict = squid_exchange_predict.groupby(['year','week']).mean()
squid_weather_predict = squid_weather_predict.groupby(['year','week']).mean()

In [103]:
# Merge Features

squid_predict = pd.DataFrame()

squid_predict['rain'] = squid_weather_predict['rain']
squid_predict['wind'] = squid_weather_predict['wind']
squid_predict['temperature'] = squid_weather_predict['temperature']
squid_predict['oil'] = df_predict['oil']
squid_predict['exchange'] = squid_exchange_predict
squid_predict['temp_kr'] = df_predict['temp_kr']
squid_predict['cpi'] = df_predict['cpi']

#### Result

In [104]:
squid_pred = squid_final.predict(squid_predict)

pd.DataFrame({'pred':np.exp(squid_pred)-1})

,pred
0,2.675492
1,2.654079
2,2.675492
3,2.665349
4,2.664072
5,2.665349
6,2.658181
7,2.665349
8,2.665349
9,2.665349


## Salmon

연어 모델용 test features들을 생성. -> 최종 결과값 도출

In [105]:
# 연어 주요 제조국 및 수출국 filtering

salmon_exchange_predict = df_exchange[(df_exchange['CTRY_2'] == '노르웨이')]
salmon_weather_predict = df_weather_with_wf[(df_weather_with_wf['CTRY_1'] == '노르웨이')]

In [106]:
# test data에 맞게 year,week filtering

salmon_exchange_predict = salmon_exchange_predict[(salmon_exchange_predict['year']==2021) & (salmon_exchange_predict['week']<=26)]
salmon_weather_predict = salmon_weather_predict[(salmon_weather_predict['year']==2021) & (salmon_weather_predict['week']<=26)]

In [107]:
# grouping

salmon_exchange_predict = salmon_exchange_predict.groupby(['year','week']).mean()
salmon_weather_predict = salmon_weather_predict.groupby(['year','week']).mean()

In [108]:
# Merge Features

salmon_predict = pd.DataFrame()

salmon_predict['rain'] = salmon_weather_predict['rain']
salmon_predict['wind'] = salmon_weather_predict['wind']
salmon_predict['temperature'] = salmon_weather_predict['temperature']
salmon_predict['oil'] = df_predict['oil']
salmon_predict['exchange'] = salmon_exchange_predict
salmon_predict['temp_kr'] = df_predict['temp_kr']
salmon_predict['cpi'] = df_predict['cpi']

#### Result

In [109]:
salmon_pred = salmon_final.predict(salmon_predict)

pd.DataFrame({'pred':np.exp(salmon_pred)-1})

,pred
0,14.756540
1,14.296228
2,14.296228
3,14.296228
4,14.294503
5,14.627148
6,14.296228
7,14.296228
8,14.296228
9,14.296228


## whiteleg_shrimp

흰다리새우 모델용 test features들을 생성. -> 최종 결과값 도출

In [110]:
# 흰다리새우 주요 제조국 및 수출국 filtering

whiteleg_shrimp_exchange_predict = df_exchange[(df_exchange['CTRY_2'] == '태국') | (df_exchange['CTRY_2'] == '베트남')]
whiteleg_shrimp_weather_predict = df_weather_with_wf[(df_weather_with_wf['CTRY_1'] == '태국') | (df_weather_with_wf['CTRY_1'] == '베트남')]

In [111]:
# test data에 맞게 year,week filtering

whiteleg_shrimp_exchange_predict = whiteleg_shrimp_exchange_predict[(whiteleg_shrimp_exchange_predict['year']==2021) & (whiteleg_shrimp_exchange_predict['week']<=26)]
whiteleg_shrimp_weather_predict = whiteleg_shrimp_weather_predict[(whiteleg_shrimp_weather_predict['year']==2021) & (whiteleg_shrimp_weather_predict['week']<=26)]

In [112]:
# grouping

whiteleg_shrimp_exchange_predict = whiteleg_shrimp_exchange_predict.groupby(['year','week']).mean()
whiteleg_shrimp_weather_predict = whiteleg_shrimp_weather_predict.groupby(['year','week']).mean()

In [113]:
# Merge Features

whiteleg_shrimp_predict = pd.DataFrame()

whiteleg_shrimp_predict['rain'] = whiteleg_shrimp_weather_predict['rain']
whiteleg_shrimp_predict['wind'] = whiteleg_shrimp_weather_predict['wind']
whiteleg_shrimp_predict['temperature'] = whiteleg_shrimp_weather_predict['temperature']
whiteleg_shrimp_predict['oil'] = df_predict['oil']
whiteleg_shrimp_predict['exchange'] = whiteleg_shrimp_exchange_predict
whiteleg_shrimp_predict['temp_kr'] = df_predict['temp_kr']
whiteleg_shrimp_predict['cpi'] = df_predict['cpi']

#### Result

In [114]:
whiteleg_shrimp_pred = whiteleg_shrimp_final.predict(whiteleg_shrimp_predict)

pd.DataFrame({'pred': np.exp(whiteleg_shrimp_pred)-1})

,pred
0,11.715026
1,11.715026
2,11.665467
3,11.715026
4,11.715026
5,11.715026
6,11.665467
7,11.665467
8,11.665467
9,11.665467
